# Import thư viện

In [4]:
# # Import libraries
# !pip install py_stringsimjoin
# !pip install py_stringmatching
import json
import os
import time
import pandas as pd
import py_stringmatching as sm
import py_stringsimjoin as ssj
from py_stringmatching.tokenizer.delimiter_tokenizer import DelimiterTokenizer
from py_stringmatching.tokenizer.qgram_tokenizer import QgramTokenizer
from itertools import product

# Define measure and tokenizer

In [5]:
# JOIN_FUNCTIONS = {'COSINE': cosine_join,
#                   'DICE': dice_join,
#                   'EDIT_DISTANCE': edit_distance_join,
#                   'JACCARD': jaccard_join,
#                   'OVERLAP': overlap_join,
#                   'OVERLAP_COEFFICIENT': overlap_coefficient_join}

TOKENIZERS = {'SPACE_DELIMITER': DelimiterTokenizer(delim_set={' '}, return_set=True),
              '2_GRAM': QgramTokenizer(qval=2, padding=False, return_set=True),
              '3_GRAM': QgramTokenizer(qval=3, padding=False, return_set=True),
              '4_GRAM': QgramTokenizer(qval=4, padding=False, return_set=True),
              '5_GRAM': QgramTokenizer(qval=5, padding=False, return_set=True),
              '2_GRAM_BAG': QgramTokenizer(qval=2),
              '3_GRAM_BAG': QgramTokenizer(qval=3)
              }

# Define filters

In [6]:
FILTERS = {
    "OVERLAP_FILTER": ssj.OverlapFilter,
    "SIZE_FILTER": ssj.SizeFilter,
    "PREFIX_FILTER": ssj.PrefixFilter,
    "POSITION_FILTER": ssj.PositionFilter,
    "SUFFIX_FILTER": ssj.SuffixFilter
}

# Define Sim_function

In [7]:
SIM_FUNC = {
    'COSINE': sm.Cosine().get_sim_score,
    'DICE': sm.Dice().get_sim_score,
    'JACCARD': sm.Jaccard().get_sim_score,
    'OVERLAP_COEFFICIENT': sm.OverlapCoefficient().get_sim_score,
    'LEVENSHTEIN': sm.Levenshtein().get_sim_score,
    'TF-IDF': sm.TfIdf,
}

# Set directory and load script

In [8]:

DATA_PATH = os.sep.join([os.getcwd(), 'dataset'])
SCRIPTS_FILE = 'scripts.json'
BENCHMARK_DIRECTORY = 'benchmark_results'

In [9]:
def load_scripts():
    with open(SCRIPTS_FILE, 'r') as js_file:
        scripts = json.load(js_file)['scripts']
    return scripts

In [133]:
def test(script, l_table, r_table, idx_script):
    result = []
    total_info_obj = product(script['sim_funcs'], script['sim_measure_types'],
                            script['tokenizers'], script['scale_filters'],
                            script['thresholds'], script['n_jobs'])
    for sim_funcs, sim_measure_type, tokenizer, scale_filter, threshold, n_jobs in total_info_obj:
        if tokenizer in ["SPACE_DELIMITER"] and sim_measure_type != 'EDIT_DISTANCE':
            continue
        sim_func = SIM_FUNC[sim_funcs]
        tok = TOKENIZERS[tokenizer]
        if scale_filter == "OVERLAP_FILTER":
            s_filter = FILTERS[scale_filter](tok, overlap_size=1, comp_op='>=', allow_missing=False)
        else:
            s_filter = FILTERS[scale_filter](tok, sim_measure_type, threshold, allow_empty=True, allow_missing=False)
        sum_time = 0
        start_time = time.time()
        candidate_set = s_filter.filter_tables(
            l_table, r_table,
            script['l_id_attr'], script['r_id_attr'],
            script['l_join_attr'], script['r_join_attr'],
            l_out_attrs=None, r_out_attrs=None,
            l_out_prefix='l_', r_out_prefix='r_',
            n_jobs=n_jobs, show_progress=False)
        output_table = ssj.apply_matcher(candidate_set,
                                        'l_' + script['l_id_attr'], 'r_' + script['r_id_attr'], l_table, r_table,
                                        script['l_id_attr'], script['r_id_attr'],
                                        script['l_join_attr'], script['r_join_attr'],
                                        tokenizer=tok, sim_function=sim_func, threshold=threshold,
                                        comp_op='>=', allow_missing=True,
                                        l_out_attrs=[script['l_join_attr']], r_out_attrs=[script['r_join_attr']],
                                        l_out_prefix='l_', r_out_prefix='r_',
                                        out_sim_score=True, n_jobs=n_jobs, show_progress=False)
        sum_time += (time.time() - start_time)
        cand_set_size = len(candidate_set)
        if not os.path.exists(BENCHMARK_DIRECTORY):
            os.makedirs(BENCHMARK_DIRECTORY)
        if not os.path.exists(os.sep.join([BENCHMARK_DIRECTORY, str(idx_script)])):
            os.makedirs(os.sep.join([BENCHMARK_DIRECTORY, str(idx_script)]))
        output_table.to_csv(os.sep.join([BENCHMARK_DIRECTORY, str(idx_script),
                            sim_measure_type + '_' + tokenizer + '_' + scale_filter + '_' + sim_funcs + '_' + str(idx_script) + 'csv']))
        result.append({"script": script, "idx_script" : idx_script, "table": output_table, "time" : sum_time, "num_candidate": cand_set_size})
    return result

In [134]:
def load_data_and_test():
    result = []
    scripts = load_scripts()
    for idx, script in enumerate(scripts):
        l_path = os.sep.join([DATA_PATH, *script['ltable']])
        r_path = os.sep.join([DATA_PATH, *script['rtable']])
        l_table = pd.read_csv(l_path, encoding=script['ltable_encoding'])
        r_table = pd.read_csv(r_path, encoding=script['rtable_encoding'])
        result.append(test(script, l_table, r_table, idx))
    return result, l_table, r_table
#result là mảng chiều: chiều thứ nhất tương ứng với chỉ số script trong list các script
#chiều thứ hai là dictionary ứng với từng lần đo trong mỗi script 

In [135]:
result, l_table, r_table = load_data_and_test()

c:\Users\Lenovo\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,
c:\Users\Lenovo\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,
c:\Users\Lenovo\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,
c:\Users\Lenovo\anaconda3\lib\site-packages\py_stringsimjoin\utils\generic_helper.py:76: FutureWarning: In a future version of pandas all arguments of DataFrame.dropna will be keyword-only.
  projected_dataframe = dataframe[proj_attrs].dropna(0,


# Đối chiếu với file DBLP-ACM_perfectMapping.csv

In [136]:
true_label_table = pd.read_csv("./dataset/library/DBLP-ACM_perfectMapping.csv", header = 0, encoding="ISO-8859-1", names = ['l_id', 'r_id'])
true_label_table

,l_id,r_id
0,conf/sigmod/SlivinskasJS01,375678
1,conf/sigmod/ChaudhuriDN01,375694
2,conf/sigmod/RinfretOO01,375669
3,conf/sigmod/BreunigKKS01,375672
4,conf/sigmod/JagadishJOT01,375687
...,...,...
2219,journals/sigmod/Scholl01,604275
2220,journals/sigmod/Rosneblatt94,190649
2221,journals/sigmod/Winslett02b,601871
2222,journals/sigmod/Labrinidis01,604283


In [137]:
import numpy as np
def caculate_true_positive(joined_table, true_label_table):
    joined_tuples = joined_table[["l_id", "r_id"]].to_records(index = False).astype(dtype=[('l_id', 'O'), ('r_id', 'O')])
    true_tuples = true_label_table[["l_id", "r_id"]].to_records(index = False).astype(dtype=[('l_id', 'O'), ('r_id', 'O')])
    return len(np.intersect1d(true_tuples, joined_tuples))


In [138]:
result[0]

[{'script': {'dataset_name': 'library',
   'ltable': ['library', 'DBLP2.csv'],
   'rtable': ['library', 'ACM.csv'],
   'ltable_encoding': 'ISO-8859-1',
   'rtable_encoding': 'ISO-8859-1',
   'l_id_attr': 'id',
   'r_id_attr': 'id',
   'l_join_attr': 'title',
   'r_join_attr': 'title',
   'tokenizers': ['3_GRAM'],
   'sim_measure_types': ['JACCARD'],
   'thresholds': [0.8],
   'n_jobs': [1],
   'scale_filters': ['POSITION_FILTER', 'SIZE_FILTER'],
   'sim_funcs': ['JACCARD']},
  'idx_script': 0,
  'table':        _id                               l_id    r_id  \
  0       33          conf/sigmod/BraumandlKK99  304573   
  1       34          conf/sigmod/WolfsonSXZC99  304572   
  2       71                conf/sigmod/Brown99  304236   
  3      211  journals/sigmod/GruenwaldBDGSSV99  333616   
  4      225       journals/sigmod/EisenbergM99  310075   
  ...    ...                                ...     ...   
  1040  7617           journals/vldb/AbbadiSW01  950482   
  1041  7618        

In [140]:
def caculate_metrics(result, true_label_table):
    for result_script in result:    
        for obj_res in result[0]:
            #true_label_table: all pairs that is actually match/true
            #obj_res["table"]: all pairs that is predicted match/true
            obj_res["true_positive"] = caculate_true_positive(obj_res["table"], true_label_table)
            obj_res["false_positive"] = len(obj_res["table"]) - obj_res["true_positive"]
            obj_res["false_negative"] = len(true_label_table) - obj_res["true_positive"]
            #base on length of candidate table, not original table
            obj_res["true_negative"] = obj_res["num_candidate"] - obj_res["true_positive"] - obj_res["false_positive"] - obj_res["false_negative"]
            obj_res["recall"] = obj_res["true_positive"] / (obj_res["true_positive"] + obj_res["false_negative"])
            obj_res["precision"] = obj_res["true_positive"] / (obj_res["true_positive"] + obj_res["false_positive"])
            obj_res["F1"] = 2 * obj_res["precision"] * obj_res["recall"] / (obj_res["precision"] + obj_res["recall"])
            obj_res["accuracy"] = (obj_res["true_positive"] + obj_res["true_negative"]) / (obj_res["true_positive"] + obj_res["true_negative"] + obj_res["false_positive"] + obj_res["false_negative"])

In [141]:
caculate_metrics(result, true_label_table)

In [142]:
result[0][0]

{'script': {'dataset_name': 'library',
  'ltable': ['library', 'DBLP2.csv'],
  'rtable': ['library', 'ACM.csv'],
  'ltable_encoding': 'ISO-8859-1',
  'rtable_encoding': 'ISO-8859-1',
  'l_id_attr': 'id',
  'r_id_attr': 'id',
  'l_join_attr': 'title',
  'r_join_attr': 'title',
  'tokenizers': ['3_GRAM'],
  'sim_measure_types': ['JACCARD'],
  'thresholds': [0.8],
  'n_jobs': [1],
  'scale_filters': ['POSITION_FILTER', 'SIZE_FILTER'],
  'sim_funcs': ['JACCARD']},
 'idx_script': 0,
 'table':        _id                               l_id    r_id  \
 0       33          conf/sigmod/BraumandlKK99  304573   
 1       34          conf/sigmod/WolfsonSXZC99  304572   
 2       71                conf/sigmod/Brown99  304236   
 3      211  journals/sigmod/GruenwaldBDGSSV99  333616   
 4      225       journals/sigmod/EisenbergM99  310075   
 ...    ...                                ...     ...   
 1040  7617           journals/vldb/AbbadiSW01  950482   
 1041  7618           journals/vldb/Atkinson